# Process 进程

**1s = 1000 ms**  
**1 ms = 1000 us**  
**1 us = 1000 ns**  

### fork 函数


> fork 是通过 写时拷贝(copy-on-write)

>源文件共享只读 -> 需要写文件时拷贝    
 
>需要写的资源 -> 自己 的虚拟地址空间

# 进程间通信


### 同一个主机




#### 匿名管道（PIPE)

- 管道----一个内核中的缓冲器，粗存能力有限，有文件的特质->读/写操作

- [^匿名管道] 没有 文件实体 ， [^有名管道]有文件实体
  - 单向的  
  - 一次性的

> 原理

<p>不同进程之间，共享<b>虚拟内存空间</b>中的文件描述符(fd[1024])</p>


        #include<unistd.h>
        int pipe(int pipefd[2]);

#### 管道的特点

- 使用管道时要注意一下集中特殊情况

  1. 所有的指向管道写端的文件描述符都被关闭了（管道写端饮用计数0），进程读数据，管道读完后，再次读read会返回0 和EOF一样；

  2. 如果有指向管道写端的fd没有关闭（>0)，而持有者也没有写数据，读端完一次后再次read  会阻塞，知道写端再次写入才会继续；

  3. 如果所有指向管道读端的文件描述符都关闭了（水管只进不出），这时候向其中写入，会受到一个信号SIGPIPE，通常进程会异常终止；

  4. 如果有指向管道读端的文件描述符没有关闭（读>0)，也没有读数据，内容没有变少。此时有人写入，如果一直写入，写满后会阻塞，直到有空位置;

>总结
>- 读管道
> - 管道中 有数据: read返回实际读到字节
> - 管道中 无数据： 
>
>              写端被全部关闭：read返回 0 （相当读到文件末尾）
>              写端没有全部关闭： read  阻塞等待
>- 写管道   
> - 读端全部关闭： 异常终止（进程收到SIGPIPE信号）
> - 读端没有完全关闭：
>
>              管道已满： write阻塞
>              管道未满： 写入并返回实际写入的数字
                
  

### 有名管道（FIFO）
>First In First Out

### 内存映射 (memory-map)

- 注意事项
 1. 可以对返回的ptr ++ -- 操作， 但是 munmap不行 ， 需要保存原地址

 2. prot的权限 <= open的权限
 
 3. 如果文件偏移量为1000会怎么样？
 
        必须是1024的整数倍，返回 MAP_FAILED

4. mmap什么情况下会调用失败？

    - 第二个参数 length = 0
    - 第三个参数 prot 
    
        - 只给了写权限
        - prot PROT_READ | PROT_WRITE  oepn制定了 只读或只写 O_RDONLY/O_WRONLY
        

5. 可以open的时候O_CREAT 一个新文件来创建映射区吗？
    - 可以，但文件大小为0，无法映射
    - 可以对文件进行扩展

        lseek()
        truncate()

6. mmap关闭文件描述符，对mmap有影响吗？

        int fd = open("XX");
        mmap(,,,,fd,0);
        close(fd);

    - 映射区还存在，fd被关闭，没有任何影响

7. 对ptr 越界操作会怎么样（野地址）？

        void * ptr = mmap(NULL,100,,,,,);
        
        越界操作是非法的内存>>段错误




### 信号 SIGNAL


#### 阻塞信号集和未决信号集

1. 用户通过键盘 CTRL+C， 产生2号信号SIGINT(信号被创建)

2. 信号产生但没有被处理（未决)

    - 在内核中将所有的没有被处理的信号，储存在一个集合中（未决信号集）
    - SIGINT 信号状态 被存储在 第二个标志位上
        - 这个标志位的值为 0 ， 说明信号不是未决状态
        - 为1， 说明信号<b>处于</b>未决状态
  <br></br>
3. 这个未决状态的信号，需要被处理，但之前需要和阻塞信号集进行比较
    
    - 阻塞信号集，默认不阻塞任何信号
    - 如果想要阻塞某系信号，需要调用API
    
    </br>
4. 在处理的时候和阻塞信号集比较，看是不是被阻塞了
    - 没有就处理
    - 阻塞就继续处于未决状态，知道阻塞解除

### 内存共享 

>效率最高的 IPC 途径

##### 相关函数
- int shmget(key_t key, size_t size, int shmflg);

    -  功能： 创建一个新的共享内存段，获取一个有烘箱内存段的表示，新创建的会初始化为0
    -  参数： 
        - key : 一个整形， 通过这这个歌找到或创建一个共享内存，一般16进制，非0值
        - size: 大小
        - shmflg: 
            - 访问权限
            - IPC_CREAT : 创建
            - IPC_EXCL ： 判断共享内存是否存在，需要和IPC_CREAT一起使用
                example: IPC_CREAT|IPC_EXCL|0664
    - 返回值:   
        - 失败 -1 
        - 成功 >0 返回共享内存的引用ID
- - - 
- void *shmat(int shmid, const void *shmaddr, int shmflg);

    - 功能： 和当前进程进行关联
    - 参数：
        - shmid: 共享内存的标识（ID)，有shmget获取
        - shmaddr: 起始地址,NULL,由内核指定
        - shmflg: 对共享内存的操作
            - 0     读写权限 
            - SHM_RDONLY     读权限（必须）
            - SHM_WEMAP
            - SHM_EXEC 
    - 返回值：          
        - 成功： 内存起始地址
        - 失败： -1
---

- int shmdt(const void *shmaddr);

    - 功能： 解除当前进程和共享内存的关联
    - 参数 首地址
    - s 0 f -1
---
- int shmctl(int shmid, int cmd, struct shmid_ds *buf);

    - 功能： 删除共享内存，共享内存只有删除了才会消失
    - 参数： 
        - shmid： 共享内存id
        - cmd ： 要做的操作
            - IPC_STAT： 获取共享区当前状态
            - IPC_SET： 设置共享内存区的状态
            - IPC_RMID : 标记结束后销毁
        - buf ：需要设置 或者获取的共享内存信息
            - IPC_STAT : buf 存储数据
            - IPC_SET : buf 初始化数据， 设置到内核中
            - IPC_RMID: 没有用 NULL
---

- key_t ftok(const char *pathname, int proj_id);

    - 功能： 根据指定的路径名，和int 值 生产一个共享内存的key
    - 参数：
        - pathname
        - proj_id ： 系统调用只会使用一个字节（8位）
                    范围 ： 0~255 一个符号

##### 内存共享问题

>问题1：操作系统如何知道一块共享内存被多少个进程关联?

        - 共享内存 维护了一个 struct shmid_ds  结构体中有一个成都 shm_nattach
        - shm_nattach 记录了关联的进程个数

>问题2： 可不可以对共享内存进行多次删除 shmctl
        
        - 可以的
        - 因为 shmctl标记删除共享内存，不是直接删除
        - 等到共享进程数为0的时候才被真正删除
        - 当共享内存的key为0的时候，共享内存被标记删除了        
                - 如果一个进程取消共享内存关联，那么这个进程就不能继续操作这个共享内存

>问题3： 共享内存 与 内存映射的区别
1. 共享内存可以 直接创建， 内存映射需要磁盘文件（匿名函数除外）
2. 共享内存效果更高
3. 内存

        - 内存共享      所有的进程操作的是同i一块内存
        - 内存映射      在自己的虚拟地址空间中有一个独立的内存
4. 数据安全

        - 进程突然退出
                共享内存：   还存在
                内存映射区：    消失了
        - 运行进程的电脑死机，宕机了
                数据存在共享内存： 没有了
                内存映射区的数据： 由于磁盘文件中的数据还在， 所以内存映射区的数据还存在


5. 生命周期


        - 内存映射区： 进程退出： 内存映射区销毁
        
        - 共享内存： 进程退出，共享内存还在-》标记删除（所有关联的进程数为0），或者关机
                        如果一个进程退出，会自动和共享内存取消关联
        